# PyTorch + PySyft

[PySyft](https://github.com/OpenMined/PySyft/), the first open-source Federated Learning framework for building secure and scalable models. PySyft is simply a hooked extension of PyTorch.

**How long does it takes to do Federated Learning compared to normal PyTorch?**

The computation time is actually less than twice the time used for normal PyTorch execution! More precisely we have a +91% overhead.

We only had to **modify 10 lines of code** to upgrade the official Pytorch example on MNIST to a real Federated Learning task!

**Currently not working on GPU**  :(


Blog: [Federated Learning and Privacy](https://dudeperf3ct.github.io/federated/learning/privacy/2019/02/08/Federated-Learning-and-Privacy/)

In [0]:
! pip install syft

    100% |████████████████████████████████| 102kB 2.8MB/s 
    100% |████████████████████████████████| 368kB 9.2MB/s 
    100% |████████████████████████████████| 204kB 27.4MB/s 
    100% |████████████████████████████████| 6.4MB 7.0MB/s 
    100% |████████████████████████████████| 92kB 26.0MB/s 
    100% |████████████████████████████████| 71kB 25.0MB/s 
    100% |████████████████████████████████| 450kB 26.8MB/s 
    100% |████████████████████████████████| 153kB 32.8MB/s 
    100% |████████████████████████████████| 102kB 29.0MB/s 
    100% |████████████████████████████████| 2.0MB 13.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/cc/42/61/e684721d69a7a5f2b8b5c1b3b77b45b98bb021df4ace993638
  Stored in directory: /root/.cache/pip/wheels/7b/6c/23/eb26369b77904c8963fae9e64338b0f0b948b4d59710760834
Successfully built zstd nodeenv


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

torch.manual_seed(42)

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print('Training on %s ...'%format(device))

Training on cuda:0 ...


## Federated Learning

![image](https://blog.openmined.org/content/images/2019/02/Capture-d-e-cran-2019-02-25-a--17.45.36.png)



Code Adapted from : [Link](https://blog.openmined.org/upgrade-to-federated-learning-in-10-lines/)

In [0]:
import syft as sy  # <-- import the Pysyft library
hook = sy.TorchHook(torch)  # <-- hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- and alice

In [0]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.log_interval = 10
        self.save_model = False

args = Arguments()

In [0]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True)

  0%|          | 16384/9912422 [00:00<01:28, 111914.41it/s]

9920512it [00:00, 29151429.82it/s]                           


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 451632.36it/s]
  1%|          | 16384/1648877 [00:00<00:11, 141886.53it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 7314635.19it/s]                           
8192it [00:00, 178942.06it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!
Scanning and sending data to bob, alice...
Done!


## CNN Model

In [0]:
# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer
        self.conv1 = nn.Conv2d(1, 32, 3, padding=0)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=0)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(p=0.25)
        self.dropout2 = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(12 * 12 * 64, 128)
        self.output = nn.Linear(128, 10)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout1(x)
        x = x.view(-1, 12 * 12 * 64)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = F.log_softmax(self.output(x), dim=1)
        return x

In [0]:
model = Net()
#model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr)

In [0]:
def train(args, model, device, train_loader, optimizer, epoch):

    model.train()
    
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
      
        model.send(data.location) # <-- send the model to the right location
        #data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- get the model back
        
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [0]:
def test(args, model, device, test_loader):
  
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
          
            #data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [0]:
for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

/usr/local/lib/python3.6/dist-packages/syft/frameworks/torch/tensors/interpreters/native.py:215: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  response = eval(cmd)(*args, **kwargs)


Train Epoch: 1 [0/60032 (0%)]	Loss: 2.290317
Train Epoch: 1 [640/60032 (1%)]	Loss: 2.274127
Train Epoch: 1 [1280/60032 (2%)]	Loss: 2.248840
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.183409
Train Epoch: 1 [2560/60032 (4%)]	Loss: 2.130820
Train Epoch: 1 [3200/60032 (5%)]	Loss: 2.016241
Train Epoch: 1 [3840/60032 (6%)]	Loss: 1.838205
Train Epoch: 1 [4480/60032 (7%)]	Loss: 1.658929
Train Epoch: 1 [5120/60032 (9%)]	Loss: 1.431365
Train Epoch: 1 [5760/60032 (10%)]	Loss: 1.111278
Train Epoch: 1 [6400/60032 (11%)]	Loss: 0.955133
Train Epoch: 1 [7040/60032 (12%)]	Loss: 1.017405
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.853076
Train Epoch: 1 [8320/60032 (14%)]	Loss: 0.836301
Train Epoch: 1 [8960/60032 (15%)]	Loss: 0.767934
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.693691
Train Epoch: 1 [10240/60032 (17%)]	Loss: 0.811199
Train Epoch: 1 [10880/60032 (18%)]	Loss: 0.532036
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.602653
Train Epoch: 1 [12160/60032 (20%)]	Loss: 0.662069
Train Epoch: 1 [12800/60032 (